# Programm zur Erkennung von Hunderassen

## Schritt 1: Daten laden und Aufteilen in Trainings- und Testdaten

In [ ]:
# Importieren der notwendigen Bibliotheken
import os
import random
import numpy as np
import tensorflow as tf
import xml.etree.ElementTree as ET
from skimage import io, transform
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import copy
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [49]:
def read_images_to_split(dataset_path, annotation_path):
    # Funktion zum Parsen der Annotationsdateien
    def parse_annotation(xml_file):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        bndboxes = []
        for obj in root.findall('object'):
            bndbox = obj.find('bndbox')
            xmin = int(float(bndbox.find('xmin').text))
            ymin = int(float(bndbox.find('ymin').text))
            xmax = int(float(bndbox.find('xmax').text))
            ymax = int(float(bndbox.find('ymax').text))
            bndboxes.append((xmin, ymin, xmax, ymax))
        return bndboxes

    # Laden der Bilder und Extrahieren der Hundebereiche
    images = []
    labels = []
    breeds = os.listdir(dataset_path)

    for breed in breeds:
        breed_img_path = os.path.join(dataset_path, breed)
        breed_anno_path = os.path.join(annotation_path, breed)
        if os.path.isdir(breed_img_path):
            for img_name in os.listdir(breed_img_path):
                img_path = os.path.join(breed_img_path, img_name)
                annotation_name = os.path.splitext(img_name)[0]
                annotation_file = os.path.join(breed_anno_path, annotation_name)
                if not os.path.exists(annotation_file):
                    continue  # Überspringe Bilder ohne Annotation
                image = io.imread(img_path)
                if image is None:
                    continue
                bndboxes = parse_annotation(annotation_file)
                for bndbox in bndboxes:
                    xmin, ymin, xmax, ymax = bndbox
                    xmin = max(0, xmin)
                    ymin = max(0, ymin)
                    xmax = min(image.shape[1], xmax)
                    ymax = min(image.shape[0], ymax)
                    if xmin >= xmax or ymin >= ymax:
                        continue  # Ungültige Bounding Box überspringen
                    cropped_image = image[ymin:ymax, xmin:xmax]
                    cropped_image = transform.resize(cropped_image, (256, 256))
                    images.append(cropped_image)
                    labels.append(breed)

    # Umwandeln der Labels in numerische Werte
    le = LabelEncoder()
    y = le.fit_transform(labels)

    # Überprüfen, ob Bilder geladen wurden
    if len(images) == 0:
        print("Fehler: Keine Bilder geladen. Bitte überprüfen Sie den Pfad und die Annotationsdateien.")

    return np.array(images), y, le


In [35]:
# Hier werden die Daten initial eingelesen, "le" ist dabei der LabelEncoder um die Labels später wieder zu den ursprünglichen Namen zu ändern
X, y, le = read_images_to_split("dogs", "annotations")

## 2. Schritt: Netzarchitektur festlegen

Convolutional Neural Networks (CNNs) eignen sich hervorragend für Bildklassifikationsaufgaben, da sie räumliche Strukturen in Bildern effektiv erkennen können. CNNs verwenden Faltungsoperationen, die es ihnen ermöglichen, lokale Merkmale wie Kanten, Texturen und Formen zu extrahieren, die für die Unterscheidung von Hunderassen essenziell sind.

**Gründe für die Wahl von CNNs:**

1. Parameterreduzierung: Durch Gewichtsteilung reduzieren CNNs die Anzahl der zu lernenden Parameter im Vergleich zu vollständig verbundenen Netzwerken. Dies ist insbesondere bei großen Eingabebildern (z. B. 256x256) wichtig.
2. Translation Invariance: CNNs können wichtige Merkmale unabhängig von deren Position im Bild erkennen, was bei der Hunderassenerkennung hilfreich ist, da die Hunde in unterschiedlichen Posen und Perspektiven dargestellt sein können.
3. Hierarchisches Lernen: CNNs lernen in den unteren Schichten einfache Merkmale (wie Kanten) und kombinieren diese in höheren Schichten zu komplexeren Merkmalen (z. B. Gesichtszüge von Hunden).

**Netzwerkarchitektur:**
Für die Hunderassenerkennung wähle ich folgende Architektur:

- *Eingabeschicht*: Input-Shape (256, 256, 3) (RGB-Bilder mit Größe 256x256).
- *Faltungsschichten (Convolutional Layers)*: Mehrere Schichten mit 3x3-Filtern und zunehmender Filteranzahl (z. B. 32, 64, 128), um unterschiedliche Merkmale zu extrahieren.
- *Pooling-Schichten (MaxPooling)*: Nach jeder Faltungsschicht zur Reduktion der räumlichen Dimension und zur Verhinderung von Überanpassung.
- *Batch-Normalisierung*: Um die Trainingsstabilität zu erhöhen und den Gradientenfluss zu verbessern.
- *Dropout*: Nach den Faltungsblöcken, um Überanpassung zu reduzieren.
- *Fully-Connected Layer*: Eine oder mehrere vollständig verbundene Schichten, um die gelernte Merkmalsrepräsentation in Klassen zu übersetzen.
- *Ausgabeschicht*: Softmax-Aktivierung mit so vielen Neuronen wie Hunderassenklassen.

Deswegen wurde nach der Input Schicht jeweils drei mal ein Convolutional Layer, gefolgt von einem MaxPooling Layer, Batch Normalization und einem Dropout Layer verwendet. 
Somit wir nach und nach erst mit der Convolutional Layer die Merkmale des Bildes extrahiert, dann mit dem MaxPooling Layer die Dimensionalität reduziert, mit der Batch Normalization die Trainingsstabilität erhöht und mit dem Dropout Layer die Überanpassung reduziert.
Am Ende wird die Flatten Schicht verwendet, um die Daten in ein Format zu bringen, welches von den Fully Connected Layern verarbeitet werden kann.
Die Ausgabeschicht hat 5 Neuronen, da es 5 Hunderassenklassen gibt.

## 3. Schritt: Netzarchitektur implementieren

In [36]:
# Netzwerkarchitektur
model = Sequential([
    # Input layer
    Input(shape=(256, 256, 3)),
    
    # Faltungsschicht 1
    Conv2D(32, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),
    # Add other layers as needed

    # Faltungsschicht 2
    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    # Faltungsschicht 3
    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),
    Dropout(0.25),

    # Übergang zu Fully-Connected Layers
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    
    # Ausgabeschicht
    Dense(5, activation='softmax')
])

# Netzwerkübersicht anzeigen
model.summary()

## 4. Schritt: Wahl einer geeigneten Lossfunktion und eines Optimizers

**Loss-Funktion:** CategoricalCrossentropy <br>
Diese Funktion eignet sich für Multi-Klassen-Klassifikationsprobleme und misst die Divergenz zwischen den vorhergesagten Wahrscheinlichkeiten und den tatsächlichen Klassenlabels.

**Optimizer:** Adam <br>
Der Adam-Optimizer kombiniert die Vorteile von AdaGrad und RMSProp und passt die Lernrate für jeden Parameter dynamisch an. Er ist robust und effizient, besonders bei großen Datenmengen und komplexen Modellen. <br>
<sup>(Quelle: Vorlesung)</sup>

In [37]:
# Kompilierung des Modells
model.compile(
    optimizer=Adam(learning_rate=0.001), # Die Learning Rate wird hier zur Standard Rate gesetzt um später den Unterschied besser sichtbar zu machen
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## 5. Evaluation mit der Kreuzvalidierung

In [ ]:
early_stopping = EarlyStopping(
    monitor='val_loss',         # Zu überwachende Metrik
    patience=5,                 # Anzahl der Epochen ohne Verbesserung, bevor das Training gestoppt wird
    restore_best_weights=True   # Beste Gewichte automatisch wiederherstellen
)

In [ ]:
# Kreuzvalidierung (3-fold)
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
fold = 1
conf_matrices = []
histories = []
all_y_true = []
all_y_pred = []

for train_idx, val_idx in kf.split(X, y):
    print(f"Fold {fold} am Laufen")
    X_train_fold, X_val_fold =X[train_idx], X[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    # One Hot Encoden der Daten
    y_train_fold = to_categorical(y_train_fold, num_classes=5)
    y_val_fold = to_categorical(y_val_fold, num_classes=5)
    
    # Training
    model.fit(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold),
              epochs=50, batch_size=32, verbose=0)
    
    # Vorhersage mit dem in diesem Fold trainiertem Modell
    y_pred = model.predict(X_val_fold)

    # Konvertieren der Werte, damit sie später ausgelesen werden können
    y_val_int = np.argmax(y_val_fold, axis=1)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # Speichern der Labels für den Report
    all_y_true.extend(y_val_int)
    all_y_pred.extend(y_pred_classes)

    
    # Speichern der Konfusionsmatrix
    conf_matrices.append(confusion_matrix(y_val_int, y_pred_classes))
    fold += 1


# Klassifizierungsbericht
print("\nKlassifizierungsbericht (Gesamtergebnis aus allen Folds):\n")
print(classification_report(all_y_true, all_y_pred, target_names=le.classes_))

# Mittelwert der Konfusionsmatrizen berechnen
avg_conf_matrix = np.mean(conf_matrices, axis=0)

# Visualisierung der Konfusionsmatrix
disp = ConfusionMatrixDisplay(confusion_matrix=avg_conf_matrix, display_labels=le.classes_)
disp.plot(cmap=plt.cm.Blues, xticks_rotation=90)
plt.title("Durchschnittliche Konfusionsmatrix")
plt.show()

### Evaluation Aufgabe 5 

#### Precision
Precision misst den Anteil der korrekt als positiv klassifizierten Instanzen an allen als positiv klassifizierten Instanzen. In diesem Bericht variieren die Precision-Werte zwischen 0.59 und 0.78, was bedeutet, dass zwischen 59% und 78% der als jeweilige Hunderasse vorhergesagten Instanzen tatsächlich korrekt waren.

#### Recall
Recall misst den Anteil der korrekt als positiv klassifizierten Instanzen an allen tatsächlich positiven Instanzen. Hier liegen die Recall-Werte zwischen 0.53 und 0.89, was zeigt, dass das Modell zwischen 53% und 89% der tatsächlichen positiven Instanzen korrekt erkannt hat.

##### F1-Score
Der F1-Score ist der harmonische Mittelwert von Precision und Recall und dient als Maß für die Balance zwischen diesen beiden Metriken. In diesem Klassifikationsreport variieren die F1-Scores zwischen 0.63 und 0.83. Ein höherer F1-Score deutet darauf hin, dass das Modell sowohl in der Genauigkeit als auch in der Erkennungsrate gut abschneidet. Hier zeigt der F1-Score, dass das Modell insgesamt eine ausgewogene Leistung erbringt, indem es sowohl falsche positive als auch falsche negative Vorhersagen minimiert. Zum Beispiel, der Maltese Dog hat den höchsten F1-Score von 0.83, was bedeutet, dass das Modell für diese Klasse besonders gut funktioniert, während der Greater Swiss Mountain Dog mit einem F1-Score von 0.63 schlechter abschneidet, was in diesem Fall an dem schlechten Recall-Wert dieser Klasse liegt.

#### Zusammenfassung
Zusammengefasst zeigt das CNN-Modell solide Leistungen bei der Klassifikation der fünf Hunderassen mit einer durchschnittlichen Genauigkeit von 0,71, was deutlich über zufälligem Raten (20%) liegt, womit das Trainieren des Modells als Erfolg gewertet werden kann. Dass Schipperke und Greater Mountain Dog schlechter abschneiden, liegt vor allem daran, dass es weniger Bilder von diesen gibt, das Gegenteil gilt für den Maltese Dog, der die meisten Bilder hat und deshalb am besten abschneidet. 

Deshalb wäre ein Ansatz  die Genauigkeit zu erhöhen, dass man Rassen mit gleich vielen Bildern wählt. Man könnte auch den ähnlichen Ansatz der Data Augmentation wählen, um eine gleiche Anzahl von Daten pro Rasse zu schaffen.
Eine weitere Möglichkeit ist die generelle Erhöhung des Datensatzes, zum Beispiel durch externe Quellen, dies würde auch ermöglichen, dass man die Daten besser evaluieren kann und das momentane Ergebnis entweder bestätigen oder widerlegen kann.
Auch wäre natürlich eine Anpassung der Modellarchitektur möglich, um zum Beispiel mehr Hunderassen zu erkennen.

## 6. Transfer Learning

In [ ]:
# Trainieren des (alten) Modells mit allen Daten, damit es für Transfer Learning genutzt werden kann
y_hot_enc = to_categorical(y, num_classes=5)
model.fit(X, y_hot_enc, epochs=30, batch_size=32, verbose=0, callbacks=[early_stopping], validation_data=(X, y_hot_enc))

In [ ]:
# Methode um den Verlauf des Trainings zu visualisieren
def visualize_history(history):

    # Loss-Werte
    loss = history['loss']
    val_loss = history['val_loss']

    # Accuracy-Werte (falls verwendet)
    accuracy = history.get('accuracy')
    val_accuracy = history.get('val_accuracy')

    # Epochen erstellen
    epochs = range(1, len(loss) + 1)

    # Plot für den Loss
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, loss, 'bo-', label='Trainings Loss')
    plt.plot(epochs, val_loss, 'ro-', label='Validierungs Loss')
    plt.title('Trainings und Validierungs Loss')
    plt.xlabel('Epochen')
    plt.ylabel('Loss')
    plt.legend()

    # Plot für die Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(epochs, accuracy, 'bo-', label='Trainings Genauigkeit')
    plt.plot(epochs, val_accuracy, 'ro-', label='Validierungs Genauigkeit')
    plt.title('Trainings und Validierungs Genauigkeit')
    plt.xlabel('Epochen')
    plt.ylabel('Genauigkeit')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [59]:
# Entfernen der letzten drei Schichten und Hinzufügen neuer Schichten

new_model = copy.deepcopy(model)

new_model.pop()
new_model.pop()
new_model.pop()

# Hinzufügen neuer Schichten
new_model.add(Dense(256, activation='relu'))
new_model.add(Dropout(0.5))
new_model.add(Dense(5, activation='softmax'))


# Gefrorene Schichten (bis zur neuen Dense-Schicht)
for layer in new_model.layers[:-3]:  # Alle außer den neuen Schichten einfrieren
    layer.trainable = False

# Kompilieren des neuen Modells
new_model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Überblick über das neue Modell
#new_model.summary()

# 
X_new, y_new, le_new = read_images_to_split("dogs_extended", "annotations")
X_train_new, X_test_new, y_train_new_spl, y_test_new_spl = train_test_split(X_new, y_new, test_size=0.25, stratify=y_new, random_state=42)

# One hot Encoden der Daten
y_train_new = to_categorical(y_train_new_spl, num_classes=5)
y_test_new = to_categorical(y_test_new_spl, num_classes=5)

print("Trainieren der neuen Ausgabeschicht")
# Training der neuen Ausgabeschicht
history = new_model.fit(
    X_train_new, y_train_new,
    validation_data=(X_test_new, y_test_new),
    epochs=30, batch_size=16, verbose=1
)

# Feinabstimmung des gesamten Modells
for layer in new_model.layers:
    layer.trainable = True  # Alle Schichten freigeben

# Reduzierte Lernrate für Fine-Tuning
new_model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print("Finetuning des gesamten Modells")
history_finetune  = new_model.fit(
    X_train_new, y_train_new,
    validation_data=(X_test_new, y_test_new),
    epochs=5, batch_size=16, verbose=1
)

# Kombinieren der Trainingsergebnisse, damit das Trainieren der letzten Ebenenen, sowie das Feintuning in einem Graph visualisiert wird
for key in history.history.keys():
    history.history[key].extend(history_finetune.history[key])

visualize_history(history.history)

In [61]:
# Vorhersagen für Testdaten
y_pred_new = new_model.predict(X_test_new)

# Konvertiere die Vorhersagen zu Klassenindizes
y_pred_classes_new = np.argmax(y_pred_new, axis=1)
y_test_int_new = np.argmax(y_test_new, axis=1)

# Konfusionsmatrix
conf_matrix_new = confusion_matrix(y_test_int_new, y_pred_classes_new)
disp_new = ConfusionMatrixDisplay(confusion_matrix=conf_matrix_new, display_labels=le_new.classes_)
disp_new.plot(cmap=plt.cm.Blues, xticks_rotation=90)
plt.title("Konfusionsmatrix - Transfer Learning")

#y_pred_classes_new = to_categorical(y_pred_classes_new)

#y_pred_classes_new = to_categorical(y_pred_classes_new)
report = classification_report(y_test_int_new, y_pred_classes_new, target_names=le_new.classes_)

print(report)


plt.show()

## Evaluation Aufgabe 6

Das Modell aus Aufgabe 5 wurde kopiert, die letzten 3 Schichten entfernt (da die Schichten davor für die Feature Extraction sind) und dann wurden drei neue Schichten hinzugefügt, welche danach mit den neuen Daten trainiert wurden, um die Zuordnung der Features zu den Klassen neu festzulegen. Danach wurde noch ein Finetuning auf das gesamte Modell angewandt.\
Trotz dessen hat das neue Modell mit den neuen Hunderassen eine schlechtere Genauigkeit (0,68(neues Modell) zu 0,71(altes Modell)), dieser Unterschied wäre eigentlich noch stärker, da bei diesem Modell die ausgewählten Hunderassen ungefähr gleich viele Bilder haben (was ja bei Aufgabe 5 nicht der Fall war).

Der Grund warum dieses Modell trotzdem schlechter abschneidet ist vermutlich unter anderem, weil die Feature Exraction auf die Hunde des ersten Modells angepasst ist. (nenne weitere Gründe)\
Auch könnte es sein, dass die neuen Hunderassen schwieriger zu erkennen sind, als die alten, was auch zu einer schlechteren Genauigkeit führen könnte, was dadurch zu beheben wäre die neuen Hunderassen nochmal genauer von den möglichen Merkmalen zu untersuchen und dann entsprechend unterschiedliche Rassen zu wählen, die sich in den Merkmalen stark unterscheiden.\
Eine weitere Möglichkeit warum dieses neue Modell schlechter abschneidet, wäre, dass die Bilder der neuen Hunderassen schlechter sind, als die der alten, also zum Beispiel die Hunde und deren Merkmale auf den Bildern nicht so gut herauskommen.